In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import os
import sys
import plotly.graph_objects as go 
dir_path = os.path.abspath('')
sys.path.append(dir_path + '/../')
from labbiofisica import final_val

In [36]:
# concentrazioni GuHCl in µM
concentrazioni = np.array([0,1.55, 3.07, 4.63, 6.20, 7.73]) # uM

In [45]:
filename = './data/fondoPBS.TXT'

fondo = pd.read_csv(filename, sep='\t', header=None, skiprows=19)  # Adjust the delimiter if necessary
fondo.columns = ['Wavelength', 'Absorbance']
fondo.tail()


coda_fondo = np.mean(fondo['Absorbance'][fondo['Wavelength'] > 270])

fondo['Absorbance'] = fondo['Absorbance'] - coda_fondo

# plt.title('fondo')
# plt.plot(fondo['Wavelength'], fondo['Absorbance'])
# plt.show()

In [46]:
fig = go.Figure()

colors = px.colors.sequential.Turbo

for i in range(6):
    #spectrum = pd.read_csv(f'./data/C{i}.TXT', sep='\t', header=None, skiprows=19)  # Adjust the delimiter if necessary
    spectrumt2 = pd.read_csv(f'./data/C{i}2.TXT', sep='\t', header=None, skiprows=19)  # Adjust the delimiter if necessary
    #spectrum.columns = ['Wavelength', 'Absorbance']
    spectrumt2.columns = ['Wavelength', 'Absorbance']
    #spectrum = spectrum.dropna()
    spectrumt2 = spectrumt2.dropna()
    
    # tolgo il fondo
    #spectrum['Absorbance'] = spectrum['Absorbance'] - fondo['Absorbance']
    spectrumt2['Absorbance'] = spectrumt2['Absorbance'] - fondo['Absorbance']

    # appaio code:
    # coda = spectrum['Absorbance'][spectrum['Wavelength'] == 340].values[0]
    coda2 = np.mean(spectrumt2['Absorbance'][spectrumt2['Wavelength'] > 270])
    # print(coda2)
    # spectrum['Absorbance'] = spectrum['Absorbance'] - coda
    spectrumt2['Absorbance'] = spectrumt2['Absorbance'] - coda2

    #fig.add_trace(go.Scatter(x=spectrum['Wavelength'], y=spectrum['Absorbance'], mode='lines', name=f'C{i}', line_color=colors[i]))
    fig.add_trace(go.Scatter(x=spectrumt2['Wavelength'], y=spectrumt2['Absorbance'], mode='lines', name=f"{concentrazioni[i]}", line_color=colors[i]))

fig.add_trace(go.Scatter(x=fondo['Wavelength'], y=fondo['Absorbance'], mode='lines', name='Fondo', line_color='red', line=dict(dash='dot')))

fig.update_layout(
    xaxis_title='λ [nm]',
    yaxis_title='Absorbance',
    legend_title='Concentration [µM]',
    width=600,
    height=400,
    legend=dict(
        x=1,
        y=1,
        xanchor='right',
        yanchor='top',
    ),
    margin=dict(l=20, r=20, t=20, b=20),
    xaxis_range=[190, 270]
)

fig.show()
